# PROTEST_DEMO: Protest tactics → EGT analysis (synthetic)

Illustrates the pipeline on protest–policing style dynamics using **tactic counts**.
Payoffs: (i) per‑capita growth of tactic counts; (ii) information‑gain toward a policy outcome.

**Caveat:** Synthetic; real analyses need event‑coded datasets, intervention windows, and strong nulls.

In [ ]:
import sys, os, numpy as np, pandas as pd, matplotlib.pyplot as plt
sys.path.append('/mnt/data')
from gameify_timeseries import growth_payoffs, info_gain_payoffs, estimate_A_from_series, find_ESS
print('Imports OK')

## 1) Simulate protest tactics with hidden archetypes and repression
Tactics: march, strike, blockade, riot. Hidden 3‑archetype mixture drives tactic shares; repression intensity modulates total events.

In [ ]:
def simulate_protest(N=4, K=3, T=260, seed=0):
    rng = np.random.default_rng(seed)
    tactics = ['march','strike','blockade','riot']
    # True tactic archetypes (nonnegative, col-normalized)
    S = rng.random((N, K))
    S /= (np.linalg.norm(S, axis=0, keepdims=True) + 1e-12)
    # Interaction among archetypes (skew → cycles)
    A_true = np.array([[0.0,  0.30, -0.20],
                       [-0.30, 0.0,  0.25],
                       [0.20, -0.25, 0.0]])
    x = np.full(K, 1.0/K)
    X_share = np.zeros((N, T))
    # Repression intensity: slow mean‑reverting + seasonality
    rep = np.zeros(T)
    r = 0.95; lvl = 0.5
    for t in range(T):
        eps = 0.1*rng.standard_normal()
        lvl = 0.8*lvl + 0.2*0.5 + eps
        rep[t] = 0.6 + 0.3*np.sin(2*np.pi*t/52) + 0.4*np.tanh(lvl)
    rep = np.clip(rep, 0.1, 1.5)
    # Simulate shares and counts
    for t in range(T):
        X_share[:, t] = np.maximum(S @ x + 0.05*rng.standard_normal(N), 1e-8)
        X_share[:, t] /= X_share[:, t].sum()
        # Replicator step on hidden archetypes
        u = A_true @ x; u -= u.mean(); dx = x * u
        x = x + 0.15*dx + 0.01*rng.random(K)
        x = np.maximum(x, 1e-8); x = x / x.sum()
    # Weekly total event rate modulated by repression (higher rep → fewer events)
    base = 500
    total = (base * np.exp(-1.2*(rep-0.6)) * (1.0 + 0.2*np.sin(2*np.pi*np.arange(T)/52)))
    # Realize integer counts with Poisson noise
    rng2 = np.random.default_rng(1)
    counts = rng2.poisson(lam=np.maximum(X_share * total, 1e-6))
    return tactics, S, X_share, counts, rep

tactics, S_true, X_share, counts, repression = simulate_protest()
N, T = counts.shape
print('counts shape:', counts.shape, 'shares shape:', X_share.shape)

## 2) Payoffs A: per‑capita growth of tactic counts

In [ ]:
v_growth = growth_payoffs(counts, dt=1.0, pad='edge')
pd.DataFrame(v_growth, index=tactics).iloc[:, :6]

## 3) Strategy basis via `nmf_on_X` on tactic shares

In [ ]:
from gameify_timeseries import nmf_on_X
# Learn nonnegative strategy archetypes
K = 3
S, H = nmf_on_X(X, k=K, iters=300, seed=1, normalize='l2')
S = S  # columns already normalized by nmf_on_X
print('S shape:', S.shape)


## 4) Fit `A` and find ESS (growth payoffs)

In [ ]:
est = estimate_A_from_series(S, X_share, v_growth, k=K, lambda_=1e-2)
A = est['A']; print('R^2 (growth):', round(est['R2'], 3))
ess = [r for r in find_ESS(A, tol=1e-8, max_support=K) if r['is_ess']]
print('ESS count:', len(ess))

## 5) Payoffs B: information‑gain toward a policy outcome
Construct a synthetic target: probability of **policy concession** next week, favoring strikes and peaceful marches, penalizing riots. Use rolling LOFO ablation.

In [ ]:
rng = np.random.default_rng(7)
w = np.array([0.5, 0.7, 0.2, -0.6])  # march, strike, blockade, riot
J = (X_share.T @ w) - 0.3*(repression - repression.mean()) + 0.1*rng.standard_normal(T)
v_info = info_gain_payoffs(X_share, J, ridge=1e-2, window=52)
est_info = estimate_A_from_series(S, X_share, v_info, k=K, lambda_=1e-2)
A_info = est_info['A']; print('R^2 (info):', round(est_info['R2'], 3))
ess_info = [r for r in find_ESS(A_info, tol=1e-8, max_support=K) if r['is_ess']]
print('ESS count (info):', len(ess_info))

## 6) Null test: circularly shift each tactic series

In [ ]:
def circular_shift_rows(M):
    N, T = M.shape
    rng = np.random.default_rng(11)
    Y = np.zeros_like(M)
    for i in range(N):
        k = int(rng.integers(0, T))
        Y[i] = np.roll(M[i], k)
    return Y

X_null = circular_shift_rows(X_share)
S_null, _ = nmf_multiplicative(X_null, r=K, iters=300, seed=5)
S_null = S_null / (np.linalg.norm(S_null, axis=0, keepdims=True) + 1e-12)
est_null = estimate_A_from_series(S_null, X_null, v_growth, k=K, lambda_=1e-2)
print('R^2 (null):', round(est_null['R2'], 3))

## 7) Quick plots

In [ ]:
plt.figure(figsize=(8,3))
for i in range(K):
    plt.plot(est['Xk'][i], label=f'x_{i+1}')
plt.title('Inferred strategy mixture x(t)'); plt.legend(); plt.show()

As, Aa = 0.5*(A + A.T), 0.5*(A - A.T)
print('||A_s||_F, ||A_a||_F =', float(np.linalg.norm(As)), float(np.linalg.norm(Aa)))